In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import time
from tqdm.notebook import tqdm
import sys
import helper_functions
sys.path.append("../../")
from src.mps import mps
from src.mps import multiplication
from src.mps import tfi_model

# Generating ground state of TFI model

In [3]:
L = 14
g = 1
J = 1
H = helper_functions.generate_TFI_Hamiltonian(L, J, g)
sparse_res = scipy.sparse.linalg.eigsh(H, 1, which='SA')
print("ground state energy:", sparse_res[0][0])
psi_0 = sparse_res[1][:, 0]

ground state energy: -17.47100405473179


In [4]:
mps_psi_0_exact = mps.MPS.initialize_from_state_vector(psi_0, L, 1000)
psi_0_compare = mps_psi_0_exact.to_state_vector()

In [5]:
print(np.all(np.isclose(psi_0, psi_0_compare)))
print(np.max(np.abs(psi_0 - psi_0_compare)))

True
2.5673907444456745e-15


# Testing if multiplying has the correct effect

In [6]:
model = tfi_model.TFIModel(14, 1., 1.)

In [11]:
mps_psi_0_exact = mps.MPS.initialize_from_state_vector(psi_0, L, 1000)

psi_1 = H@psi_0
mps_psi_1_exact = multiplication.multiply(mps_psi_0_exact, model.H_mpo, 1000, 0, inplace=False, compress=False)
psi_1_compare = mps_psi_1_exact.to_state_vector()

In [12]:
print(np.all(np.isclose(psi_1, psi_1_compare)))
print(np.max(np.abs(psi_1 - psi_1_compare)))

True
4.085620730620576e-14


In [15]:
psi_all_up = np.zeros(2**L)
psi_all_up[0] = 1.

In [16]:
mps_psi_0_exact = mps.MPS.initialize_from_state_vector(psi_all_up, L, 1000)

psi_1 = H@psi_all_up
mps_psi_1_exact = multiplication.multiply(mps_psi_0_exact, model.H_mpo, 1000, 0, inplace=False, compress=False)
psi_1_compare = mps_psi_1_exact.to_state_vector()

In [17]:
print(np.all(np.isclose(psi_1, psi_1_compare)))
print(np.max(np.abs(psi_1 - psi_1_compare)))

True
0.0


# Testing if normalizing works as expected

In [19]:
mps_psi_1_exact = multiplication.multiply(mps_psi_0_exact, model.H_mpo, 1000, 0, inplace=False, compress=True)

In [20]:
print(helper_functions.overlap(mps_psi_1_exact, mps_psi_1_exact))

1.0000000000000009


In [21]:
print(helper_functions.overlap(mps_psi_0_exact, mps_psi_0_exact))

1.0


In [22]:
mps_psi_1_exact.compress(3, 0)
print(helper_functions.overlap(mps_psi_1_exact, mps_psi_1_exact))

0.9999999999999999
